In [9]:
import cv2
import imageio
import math
import numpy as np
from cvzone import HandTrackingModule
import mediapipe as mp

# Khởi tạo webcam
cap = cv2.VideoCapture(0)  
cap.set(3, 1080)
cap.set(4, 720)

selfie_segmentation = mp.solutions.selfie_segmentation.SelfieSegmentation(model_selection=1)
detector = HandTrackingModule.HandDetector(maxHands=2, detectionCon=0.5)


def replace_background(image, results_selfie):
    # Đọc video/hình ảnh làm nền
    background_image = cv2.imread("Dark_background.jpg")

    # Resize background image to match the size of the input image
    background_image = cv2.resize(background_image, (image.shape[1], image.shape[0]))

    # Lấy mask kết quả từ Selfie Segmentation
    mask = results_selfie.segmentation_mask

    # Áp dụng hiệu ứng nền
    condition = np.stack((mask,) * 3, axis=-1) > 0.7
    image = np.where(condition, image, background_image)

    return image



# Hàm tạo effect thứ nhất
def sticker1(frame, effect0, effect1, hands):
    if len(hands) == 2:
        return frame
    hand = hands[0]
    if hand:
        thump_tip = hand["lmList"][4][:2]
        pinky_mcp = hand["lmList"][17][:2]
        index_finger_tip = hand["lmList"][8][:2]
        distance = math.sqrt((pinky_mcp[0] - thump_tip[0]) ** 2 + (pinky_mcp[1] - thump_tip[1]) ** 2)

        # Vẽ hình tròn tại thumb tip
        cv2.circle(frame, thump_tip, 5, (255, 255, 0), -1)
        effect0_copy = cv2.resize(effect0, (int(distance), int(distance)))
        effect1_copy = cv2.resize(effect1, (int(distance), int(distance)))
        if thump_tip[0] - index_finger_tip[0] > 0:
            x_min = int(index_finger_tip[0])
            y_min = int(thump_tip[1])
            x_max = int(x_min + distance)
            y_max = int(y_min + distance)

            # Tạo matrix transform từ 4 điểm của sticker và frame
            pts_sticker = np.float32([[0, 0], [effect0_copy.shape[1], 0], [0, effect0_copy.shape[0]], [effect0_copy.shape[1], effect0_copy.shape[0]]])
            pts_frame = np.float32([[x_min, y_min], [x_max, y_min], [x_min, y_max], [x_max, y_max]])
            matrix = cv2.getPerspectiveTransform(pts_sticker, pts_frame)

            # Áp dụng matrix transform để đưa sticker vào frame
            sticker_warped = cv2.warpPerspective(effect0_copy, matrix, (frame.shape[1], frame.shape[0]))

            # Tạo mask từ sticker_warped để áp dụng blend
            sticker_mask = cv2.cvtColor(sticker_warped, cv2.COLOR_BGR2GRAY)
            _, sticker_mask = cv2.threshold(sticker_mask, 1, 255, cv2.THRESH_BINARY)

            # Áp dụng blend
            foreground = cv2.bitwise_and(sticker_warped, sticker_warped, mask=sticker_mask)
            background = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(sticker_mask))
            frame = cv2.add(foreground, background)
        else:
            x_min = int(thump_tip[0])
            y_min = int(thump_tip[1])
            x_max = int(x_min + distance)
            y_max = int(y_min + distance)

            # Tạo matrix transform từ 4 điểm của sticker và frame
            pts_sticker = np.float32([[0, 0], [effect1_copy.shape[1], 0], [0, effect1_copy.shape[0]], [effect1_copy.shape[1], effect1_copy.shape[0]]])
            pts_frame = np.float32([[x_min, y_min], [x_max, y_min], [x_min, y_max], [x_max, y_max]])
            matrix = cv2.getPerspectiveTransform(pts_sticker, pts_frame)

            # Áp dụng matrix transform để đưa sticker vào frame
            sticker_warped = cv2.warpPerspective(effect1_copy, matrix, (frame.shape[1], frame.shape[0]))

            # Tạo mask từ sticker_warped để áp dụng blend
            sticker_mask = cv2.cvtColor(sticker_warped, cv2.COLOR_BGR2GRAY)
            _, sticker_mask = cv2.threshold(sticker_mask, 1, 255, cv2.THRESH_BINARY)

            # Áp dụng blend
            foreground = cv2.bitwise_and(sticker_warped, sticker_warped, mask=sticker_mask)
            background = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(sticker_mask))
            frame = cv2.add(foreground, background)

    return frame


def sticker2(frame, effect, hands):
    if len(hands) != 2:
        return frame
    hand_left = hands[0]
    hand_right = hands[1]
    w, h = frame.shape[:2]
    if hand_left and hand_right:
        pinky_mcp_left = hand_left["lmList"][17][:2]
        pinky_mcp_right = hand_right["lmList"][17][:2]

        # Vẽ hình tròn tại pinky tip trái và phải
        cv2.circle(frame, pinky_mcp_left, 5, (255, 0, 255), -1)
        cv2.circle(frame, pinky_mcp_right, 5, (255, 0, 255), -1)

        # Tính khoảng cách giữa hai tọa độ pinky tip
        distance = math.sqrt((pinky_mcp_right[0] - pinky_mcp_left[0]) ** 2 + (pinky_mcp_right[1] - pinky_mcp_left[1]) ** 2)

        effect_copy = cv2.resize(effect, (int(distance), int(distance)))

        center_x = int((pinky_mcp_left[0] + pinky_mcp_right[0]) / 2)
        center_y = int((pinky_mcp_left[1] + pinky_mcp_right[1]) / 2)

        x_min = int(center_x - distance / 2)
        y_min = int(center_y - distance / 2)
        x_max = int(x_min + distance)
        y_max = int(y_min + distance)


        # Tạo matrix transform từ 4 điểm của sticker và frame
        pts_sticker = np.float32([[0, 0], [effect_copy.shape[1], 0], [0, effect_copy.shape[0]], [effect_copy.shape[1], effect_copy.shape[0]]])
        pts_frame = np.float32([[x_min, y_min], [x_max, y_min], [x_min, y_max], [x_max, y_max]])
        matrix = cv2.getPerspectiveTransform(pts_sticker, pts_frame)

        # Áp dụng matrix transform để đưa sticker vào frame
        sticker_warped = cv2.warpPerspective(effect_copy, matrix, (frame.shape[1], frame.shape[0]))

        # Tạo mask từ sticker_warped để áp dụng blend
        sticker_mask = cv2.cvtColor(sticker_warped, cv2.COLOR_BGR2GRAY)
        _, sticker_mask = cv2.threshold(sticker_mask, 1, 255, cv2.THRESH_BINARY)

        # Áp dụng blend
        foreground = cv2.bitwise_and(sticker_warped, sticker_warped, mask=sticker_mask)
        background = cv2.bitwise_and(frame, frame, mask=cv2.bitwise_not(sticker_mask))
        frame = cv2.add(foreground, background)

    return frame

# ... (phần mã tiếp theo)

gif0 = imageio.mimread("shot1-unscreen.gif")
gif1 = imageio.mimread("Hearts-shot.gif")
gif2 = imageio.mimread("IU.gif")
index0 = 0
index1 = 0
index2 = 0
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Không thể đọc từ nguồn")


    # Thực hiện xử lý nhận dạng bàn tay
    hands = detector.findHands(frame, draw=False)
    selfie = selfie_segmentation.process(frame)
    frame = replace_background(frame, selfie)
    if len(hands) > 0:
        if len(hands) == 1:
            gif0_frame = gif0[index0]
            index0 = (index0+1)%len(gif0)
            gif0_frame = cv2.resize(gif0_frame, (frame.shape[1], frame.shape[0]))
            gif0_frame = cv2.cvtColor(gif0_frame, cv2.COLOR_BGR2RGB)
            gif1_frame = gif1[index1]
            index1 = (index1+1)%len(gif1)
            gif1_frame = cv2.resize(gif1_frame, (frame.shape[1], frame.shape[0]))
            gif1_frame = cv2.cvtColor(gif1_frame, cv2.COLOR_BGR2RGB)
            frame = sticker1(frame, gif0_frame, gif1_frame, hands)
        elif len(hands) == 2:
            gif2_frame = gif2[index2]
            index2 = (index2+1)%len(gif2)
            gif2_frame = cv2.resize(gif2_frame, (frame.shape[1], frame.shape[0]))
            gif2_frame = cv2.cvtColor(gif2_frame, cv2.COLOR_BGR2RGB)
            frame = sticker2(frame, gif2_frame, hands)
    else:
        pass
    # Hiển thị hình ảnh
    frame = cv2.flip(frame, 1)
    cv2.imshow("Filter Instagram", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Khi kết thúc, giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\core\src\arithm.cpp:214: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and type), nor 'array op scalar', nor 'scalar op array' in function 'cv::binary_op'
